# Grover's Algorithm: Quantum Search Demonstration

Welcome to this interactive demonstration of **Grover's Algorithm**, one of the most important quantum algorithms that provides a quadratic speedup for searching unsorted databases.

## Learning Objectives

By the end of this notebook, you will understand:
- The mathematical foundations of Grover's algorithm
- How quantum superposition and interference enable speedup
- The structure of the quantum circuit
- Practical implementation using Qiskit
- Performance comparison with classical search

## Algorithm Overview

**Grover's Algorithm** searches an unsorted database of N items in approximately √N steps, compared to the classical requirement of N/2 steps on average.

**Key Components:**
- **Oracle Function**: Marks the target item by flipping its amplitude
- **Diffusion Operator**: Amplifies the marked item's probability
- **Iterations**: Repeat Oracle + Diffusion approximately π√N/4 times

**Mathematical Representation:**
|ψ⟩ = α|target⟩ + β|other states⟩

Where after optimal iterations, |α|² ≈ 1 (high probability of measuring the target).

In [ ]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile, assemble
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector
import math

# Set up matplotlib for better plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")
print(f"Using Qiskit version: {qiskit.__version__}")

## Mathematical Foundations

### Quantum Superposition
Grover's algorithm begins by creating a uniform superposition of all possible states:

$$|\psi\rangle = \frac{1}{\sqrt{N}} \sum_{x=0}^{N-1} |x\rangle$$

For n qubits, we have N = 2ⁿ possible states.

### Oracle Function
The oracle O marks the target item by flipping its phase:
- O|target⟩ = -|target⟩  
- O|other⟩ = |other⟩

### Diffusion Operator (Amplitude Amplification)
The diffusion operator D reflects amplitudes about the average:

$$D = 2|\psi\rangle\langle\psi| - I$$

### Optimal Number of Iterations
The optimal number of iterations is approximately:

$$k_{optimal} = \frac{\pi}{4}\sqrt{N}$$

This gives success probability ≈ 1 for finding the marked item.

In [ ]:
def create_oracle(target_item, num_qubits):
    """
    Create an oracle that marks the target item by flipping its phase.
    
    Args:
        target_item (int): The item to mark (0 to 2^num_qubits - 1)
        num_qubits (int): Number of qubits
    
    Returns:
        QuantumCircuit: Oracle circuit
    """
    oracle = QuantumCircuit(num_qubits, name='Oracle')
    
    # Convert target to binary representation
    target_binary = format(target_item, f'0{num_qubits}b')
    
    # Apply X gates to qubits that should be 0 in the target state
    for i, bit in enumerate(target_binary):
        if bit == '0':
            oracle.x(i)
    
    # Multi-controlled Z gate (phase flip)
    if num_qubits > 1:
        oracle.h(num_qubits - 1)
        oracle.mcx(list(range(num_qubits - 1)), num_qubits - 1)
        oracle.h(num_qubits - 1)
    else:
        oracle.z(0)
    
    # Undo X gates
    for i, bit in enumerate(target_binary):
        if bit == '0':
            oracle.x(i)
    
    return oracle

# Test the oracle
target = 3  # Target item |011⟩
n_qubits = 3
oracle_circuit = create_oracle(target, n_qubits)
print(f"Oracle circuit for target item {target} (|{format(target, f'0{n_qubits}b')}⟩):")
print(oracle_circuit)

In [ ]:
def create_diffusion_operator(num_qubits):
    """
    Create the diffusion operator for Grover's algorithm.
    This operator reflects amplitudes about the average amplitude.
    
    Args:
        num_qubits (int): Number of qubits
    
    Returns:
        QuantumCircuit: Diffusion operator circuit
    """
    diffusion = QuantumCircuit(num_qubits, name='Diffusion')
    
    # Apply H gates
    diffusion.h(range(num_qubits))
    
    # Apply X gates
    diffusion.x(range(num_qubits))
    
    # Multi-controlled Z gate
    if num_qubits > 1:
        diffusion.h(num_qubits - 1)
        diffusion.mcx(list(range(num_qubits - 1)), num_qubits - 1)
        diffusion.h(num_qubits - 1)
    else:
        diffusion.z(0)
    
    # Apply X gates
    diffusion.x(range(num_qubits))
    
    # Apply H gates
    diffusion.h(range(num_qubits))
    
    return diffusion

# Test the diffusion operator
diffusion_circuit = create_diffusion_operator(3)
print("Diffusion operator circuit:")
print(diffusion_circuit)

In [ ]:
def grovers_algorithm(target_item, num_qubits, num_iterations=None):
    """
    Implement Grover's algorithm for quantum search.
    
    Args:
        target_item (int): The item to search for
        num_qubits (int): Number of qubits (search space size = 2^num_qubits)
        num_iterations (int): Number of Grover iterations (if None, use optimal)
    
    Returns:
        QuantumCircuit: Complete Grover's algorithm circuit
    """
    # Calculate optimal iterations if not specified
    if num_iterations is None:
        N = 2 ** num_qubits
        num_iterations = int(math.pi * math.sqrt(N) / 4)
    
    # Create quantum circuit
    qreg = QuantumRegister(num_qubits, 'q')
    creg = ClassicalRegister(num_qubits, 'c')
    circuit = QuantumCircuit(qreg, creg)
    
    # Initialize superposition
    circuit.h(qreg)
    circuit.barrier()
    
    # Grover iterations
    oracle = create_oracle(target_item, num_qubits)
    diffusion = create_diffusion_operator(num_qubits)
    
    for i in range(num_iterations):
        circuit.compose(oracle, inplace=True)
        circuit.barrier()
        circuit.compose(diffusion, inplace=True)
        circuit.barrier()
    
    # Measurement
    circuit.measure(qreg, creg)
    
    return circuit, num_iterations

# Create Grover's circuit for searching item 3 in 8-item database
target_item = 3
num_qubits = 3
grover_circuit, iterations = grovers_algorithm(target_item, num_qubits)

print(f"Grover's algorithm circuit:")
print(f"Target item: {target_item} (|{format(target_item, f'0{num_qubits}b')}⟩)")
print(f"Search space: {2**num_qubits} items")
print(f"Optimal iterations: {iterations}")
print(f"Circuit depth: {grover_circuit.depth()}")

In [ ]:
# Simulate Grover's algorithm
def simulate_grover(circuit, shots=1024):
    """Simulate Grover's circuit and return results."""
    simulator = AerSimulator()
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return counts

# Run simulation
simulation_results = simulate_grover(grover_circuit, shots=1024)

# Calculate success probability
target_binary = format(target_item, f'0{num_qubits}b')
success_count = simulation_results.get(target_binary, 0)
success_probability = success_count / 1024

print(f"Simulation Results:")
print(f"Target state |{target_binary}⟩ measured {success_count} times out of 1024")
print(f"Success probability: {success_probability:.3f}")
print(f"Classical random search success probability: {1/8:.3f}")
print(f"Quantum speedup factor: {success_probability / (1/8):.1f}x")

# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Measurement histogram
states = list(simulation_results.keys())
counts = list(simulation_results.values())
colors = ['red' if state == target_binary else 'blue' for state in states]

ax1.bar(states, counts, color=colors, alpha=0.7)
ax1.set_xlabel('Measured States')
ax1.set_ylabel('Count')
ax1.set_title('Grover\'s Algorithm Measurement Results')
ax1.tick_params(axis='x', rotation=45)

# Success probability comparison
methods = ['Classical\n(Random)', 'Quantum\n(Grover)']
probabilities = [1/8, success_probability]
bars = ax2.bar(methods, probabilities, color=['gray', 'green'], alpha=0.7)
ax2.set_ylabel('Success Probability')
ax2.set_title('Search Algorithm Comparison')
ax2.set_ylim(0, 1)

# Add value labels on bars
for bar, prob in zip(bars, probabilities):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{prob:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()